In [441]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import os
import xarray as xr
import datetime
#批量读文件还没写完，暂时没用
train_dir = '../tcdata/enso_round1_train_20210201'
for i in os.listdir(train_dir):
    if i.endswith('.nc'):
        x = nc.Dataset(train_dir + '/' + i)
        print(x)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): year(4645), month(36)
    variables(dimensions): float64 nino(year, month), int32 year(year), int32 month(month)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): year(4645), month(36), lat(24), lon(72)
    variables(dimensions): float64 sst(year, month, lat, lon), float64 t300(year, month, lat, lon), float64 ua(year, month, lat, lon), float64 va(year, month, lat, lon), int32 year(year), int32 month(month), float32 lat(lat), float32 lon(lon)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): year(100), month(36)
    variables(dimensions): float64 nino(year, month), int32 year(year), int32 month(month)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): year(100), month(

In [442]:
#读读CIMP_lable信息  先把train中的数据转换到dataframe中再加上lable里的nino
a=nc.Dataset('../tcdata/enso_round1_train_20210201/CMIP_label.nc')
dica={}
for i in a.variables.keys():
    dica[i]=a.variables[i][:].data

In [443]:
A= xr.open_dataset(xr.backends.NetCDF4DataStore(a))
A

<xarray.Dataset>
Dimensions:  (month: 36, year: 4645)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 ... 4639 4640 4641 4642 4643 4644 4645
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
Data variables:
    nino     (year, month) float64 ...

In [444]:
dica

{'nino': array([[-0.26102549, -0.13325377, -0.01483156, ...,  0.70016259,
          0.69692588,  0.62087137],
        [-0.51252842, -0.67330021, -0.69312692, ..., -3.32816887,
         -3.33858109, -3.00591755],
        [-0.16938901,  0.16518302,  0.42523685, ..., -0.21898462,
         -0.15724176, -0.03204625],
        ...,
        [-0.16746233,  0.02371221,  0.19209623, ..., -0.45745715,
         -0.4982231 , -0.43920282],
        [ 0.63583368,  0.6991865 ,  0.82033938, ..., -0.66224277,
         -0.65602088, -0.60984963],
        [ 0.65862387,  0.48303536,  0.41316095, ...,  0.56690937,
          0.63597536,  0.73177081]]),
 'year': array([   1,    2,    3, ..., 4643, 4644, 4645], dtype=int32),
 'month': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36], dtype=int32)}

In [445]:
b=nc.Dataset('../tcdata/enso_round1_train_20210201/CMIP_train.nc')
dicb={}
for i in b.variables.keys():
    dicb[i]=b.variables[i][:]

In [446]:
dicb.keys()

dict_keys(['sst', 't300', 'ua', 'va', 'year', 'month', 'lat', 'lon'])

In [507]:
B= xr.open_dataset(xr.backends.NetCDF4DataStore(b))

In [519]:
CMIP=B
CMIP['nino']=(('year','month'),A['nino'])

In [520]:
CMIP=CMIP.drop_sel(month=np.arange(13,37))

In [521]:
#下面先分割模式再拼时间
#CMIP.expand_dims("mode")

In [522]:
CMIPdf=CMIP.to_dataframe()

In [528]:
CMIPdf.head()

sst      t300        ua        va      nino
lat   lon month year                                                  
-55.0 0.0 1     1     0.346436 -0.244890 -1.680264 -0.489486 -0.261025
                2    -0.493941 -0.165055  2.115858  0.994749 -0.512528
                3    -0.018142 -0.261186 -1.463467  1.306921 -0.169389
                4     1.050851  0.251320  0.169948  0.310134  0.363714
                5     0.449654  0.393005  4.141575 -0.783133 -2.557616

In [533]:
CMIPdf.loc[(-55.0,0,1,1)]

sst     0.346436
t300   -0.244890
ua     -1.680264
va     -0.489486
nino   -0.261025
Name: (-55.0, 0.0, 1, 1), dtype: float64

In [550]:
CMIPdf=CMIPdf.reset_index()
CMIPdf.head()

,lat,lon,month,year,mode,sst,t300,ua,va,nino,day
0,-55.0,0.0,1,1,60001,0.346436,-0.244890,-1.680264,-0.489486,-0.261025,1
1,-55.0,0.0,1,2,60001,-0.493941,-0.165055,2.115858,0.994749,-0.512528,1
2,-55.0,0.0,1,3,60001,-0.018142,-0.261186,-1.463467,1.306921,-0.169389,1
3,-55.0,0.0,1,4,60001,1.050851,0.251320,0.169948,0.310134,0.363714,1
4,-55.0,0.0,1,5,60001,0.449654,0.393005,4.141575,-0.783133,-2.557616,1


In [535]:
def trans(x):
    temp=10000*(6-(x>2265))+(x<2266)*((x-1)//151+1)+(x>2265)*(((x-1)-2265)//140+1)#这里返回str报错
    #<2266:CMIP6 >2265:CMIP5
    #编码顺序开始5，6，1表示CMIP5,6,SODA 后面表示模式 从1计数 
    return temp
CMIPdf['mode']=CMIPdf['year'].apply(trans)

In [538]:
CMIPdf.head()

,lat,lon,month,year,sst,t300,ua,va,nino,mode
0,-55.0,0.0,1,1,0.346436,-0.244890,-1.680264,-0.489486,-0.261025,60001
1,-55.0,0.0,1,2,-0.493941,-0.165055,2.115858,0.994749,-0.512528,60001
2,-55.0,0.0,1,3,-0.018142,-0.261186,-1.463467,1.306921,-0.169389,60001
3,-55.0,0.0,1,4,1.050851,0.251320,0.169948,0.310134,0.363714,60001
4,-55.0,0.0,1,5,0.449654,0.393005,4.141575,-0.783133,-2.557616,60001


In [539]:
def transyear(x):#因为遍历规则，不能对X比较大小 报错ambiguous
    temp=(x<2266)*((x-1)%151+1)+(x>2265)*((x-1-2265)%140+1)#年份的顺序，计数从1开始
    return temp
CMIPdf['year']=CMIPdf['year'].apply(transyear)
CMIPdf.head()

,lat,lon,month,year,sst,t300,ua,va,nino,mode
0,-55.0,0.0,1,1,0.346436,-0.244890,-1.680264,-0.489486,-0.261025,60001
1,-55.0,0.0,1,2,-0.493941,-0.165055,2.115858,0.994749,-0.512528,60001
2,-55.0,0.0,1,3,-0.018142,-0.261186,-1.463467,1.306921,-0.169389,60001
3,-55.0,0.0,1,4,1.050851,0.251320,0.169948,0.310134,0.363714,60001
4,-55.0,0.0,1,5,0.449654,0.393005,4.141575,-0.783133,-2.557616,60001


In [557]:
#CMIPdf=CMIPdf.drop('day',axis=1)

In [558]:
CMIPdf['time']=CMIPdf.year*12+CMIPdf.month-12

In [542]:
#这一步之后可转换为Dataset
#CMIPdf = CMIPdf.set_index(['lat', 'lon','month','year','mode'])

In [545]:
#datetime.datetime(year=2000,month=1,day=1)

datetime.datetime(2000, 1, 1, 0, 0)

In [560]:
c=nc.Dataset('../tcdata/enso_round1_train_20210201/SODA_train.nc')
C= xr.open_dataset(xr.backends.NetCDF4DataStore(c))
d=nc.Dataset('../tcdata/enso_round1_train_20210201/SODA_label.nc')
D= xr.open_dataset(xr.backends.NetCDF4DataStore(d))
SODA=C
SODA['nino']=(('year','month'),D.nino)
SODA=SODA.drop_sel(month=np.arange(13,37))
SODAdf=SODA.to_dataframe()

In [564]:
SODAdf=SODAdf.reset_index()

In [565]:
SODAdf['mode']=10000

In [566]:
SODAdf['time']=SODAdf.year*12+SODAdf.month-12

In [568]:
SODAdf.head()

,lat,lon,month,year,sst,t300,ua,va,nino,mode,time
0,-55.0,0.0,1,1,1.861851,0.374280,0.429292,0.087189,-0.407207,10000,1
1,-55.0,0.0,1,2,-0.272264,0.062039,2.033018,-0.978357,0.066972,10000,13
2,-55.0,0.0,1,3,0.029107,0.023624,0.014763,-0.337003,-0.732018,10000,25
3,-55.0,0.0,1,4,0.116086,0.002191,-0.779131,0.283343,-0.259308,10000,37
4,-55.0,0.0,1,5,-0.303765,-0.017604,0.718747,0.432904,-0.123334,10000,49


In [570]:
data=pd.concat([CMIPdf, SODAdf])

In [572]:
data_reindex=data.set_index(['lat', 'lon','month','year','mode'])

In [576]:
data_set=xr.Dataset.from_dataframe(data_reindex)

In [577]:
data_set

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, mode: 33, month: 12, year: 151)
Coordinates:
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year     (year) int64 1 2 3 4 5 6 7 8 9 ... 144 145 146 147 148 149 150 151
  * mode     (mode) int64 10000 50001 50002 50003 ... 60012 60013 60014 60015
Data variables:
    index    (lat, lon, month, year, mode) float64 nan 2.265e+03 ... 9.632e+07
    sst      (lat, lon, month, year, mode) float64 1.862 -0.3493 ... 0.252
    t300     (lat, lon, month, year, mode) float64 0.3743 -0.7049 ... 0.2332
    ua       (lat, lon, month, year, mode) float64 0.4293 -2.984 ... -1.701
    va       (lat, lon, month, year, mode) float64 0.08719 0.07952 ... 2.318
    nino     (lat, lon, month, year, mode) float64 -0.4072 -1.91 ... -0.5696
    time     (lat, lon, month, year, mode) float64 1.0 1.0 ... 1.812e+03

In [581]:
data_set.to_netcdf("notebook/data_set.nc")

In [ ]:
data.to_csv("notebook/data_frame.csv")

## XR
def trans(x):
    temp=10000*(6-(x>2265))+(x<2266)*((x-1)//151+1)+(x>2265)*(((x-1)-2265)//140+1)#这里返回str报错
    #<2266:CMIP6 >2265:CMIP5
    #编码顺序开始5，6，1表示CMIP5,6,SODA 后面表示模式 从1计数 
    return temp
CMIP=CMIP.assign_coords(mode=("mode",trans(CMIP.year)))
CMIP

ValueError: Dimension mode already exists.

In [490]:
CMIP

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, mode: 4645, time1: 55740)
Coordinates:
  * lat      (lat) float32 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float32 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * mode     (mode) int64 60001 60001 60001 60001 ... 50017 50017 50017 50017
    time     (time1) int32 1 2 3 4 5 6 7 ... 1674 1675 1676 1677 1678 1679 1680
  * time1    (time1) MultiIndex
  - year     (time1) int64 1 1 1 1 1 1 1 1 1 ... 140 140 140 140 140 140 140 140
  - month    (time1) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    sst      (lat, lon, time1) float64 0.3464 0.222 0.3988 ... 0.5441 0.6906
    t300     (lat, lon, time1) float64 -0.2449 -0.2425 -0.1634 ... 0.5012 0.5558
    ua       (lat, lon, time1) float64 -1.68 2.064 0.2488 ... 1.135 3.162 4.196
    va       (lat, lon, time1) float64 -0.4895 -2.865 -3.345 ... 2.361 -0.5992
    nino     (time1) float64 -0.261 -0.1333 -0.01483 ... -0.4575 -0.4982 -0.4392

def transyear(x):#因为遍历规则，不能对X比较大小 报错ambiguous
    temp=(x<2266)*((x-1)%151+1)+(x>2265)*((x-1-2265)%140+1)#年份的顺序，计数从1开始
    return temp
CMIP['year']=CMIP.year.pipe(transyear)
CMIP

In [479]:
#CMIP=CMIP.stack(time=["year", "month"])#datetime不适用，因为没有日期，且修改coords时参数为tuple时pipe报错

CMIP['time']=CMIP.year*12+CMIP.month-12

#CMIP=CMIP.assign_coords(time=("time",CMIP.year*12+CMIP.month-12))
CMIP=CMIP.set_coords('time')
CMIP

#CMIP.load()
#CMIP=CMIP.persist()
#CMIP=CMIP.stack(time1=["year",'month'])#去除标签重复值
CMIP=CMIP.unstack("time1")
CMIP

In [432]:
#CMIP.load()
#CMIP=CMIP.persist()

In [372]:
#xr.concat([CMIP.sel(mode=i) for i in CMIP.mode.values], "time")报错了
#CMIP.time=CMIP.time.pipe(lambda x:datetime(x+(1,0,0))) 报错了

In [374]:
c=nc.Dataset('../tcdata/enso_round1_train_20210201/SODA_train.nc')
C= xr.open_dataset(xr.backends.NetCDF4DataStore(c))
C

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, month: 36, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
Data variables:
    sst      (year, month, lat, lon) float32 ...
    t300     (year, month, lat, lon) float32 ...
    ua       (year, month, lat, lon) float64 ...
    va       (year, month, lat, lon) float64 ...

In [375]:
d=nc.Dataset('../tcdata/enso_round1_train_20210201/SODA_label.nc')
D= xr.open_dataset(xr.backends.NetCDF4DataStore(d))
D

<xarray.Dataset>
Dimensions:  (month: 36, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
Data variables:
    nino     (year, month) float64 ...

In [377]:
SODA=C
SODA['nino']=(('year','month'),D.nino)
SODA

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, month: 36, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
Data variables:
    sst      (year, month, lat, lon) float32 ...
    t300     (year, month, lat, lon) float32 ...
    ua       (year, month, lat, lon) float64 ...
    va       (year, month, lat, lon) float64 ...
    nino     (year, month) float64 -0.4072 -0.2024 -0.1039 ... 1.64 1.557 1.38

In [382]:
SODA=SODA.drop_sel(month=np.arange(13,37))

In [385]:
SODA=SODA.assign_coords(mode=("mode",10000*(SODA.year>0)))

In [388]:
SODA['time']=SODA.year*12+SODA.month-12
SODA=SODA.set_coords('time')
SODA

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, mode: 100, month: 12, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * mode     (mode) int64 10000 10000 10000 10000 ... 10000 10000 10000 10000
    time     (year, month) int32 1 2 3 4 5 6 7 ... 1195 1196 1197 1198 1199 1200
Data variables:
    sst      (year, month, lat, lon) float32 ...
    t300     (year, month, lat, lon) float32 ...
    ua       (year, month, lat, lon) float64 ...
    va       (year, month, lat, lon) float64 ...
    nino     (year, month) float64 -0.4072 -0.2024 -0.1039 ... -1.177 -1.365

In [401]:
SODA=SODA.stack(time2=["year","month"])#去除标签重复值
SODA=SODA.unstack("time2")
SODA

<xarray.Dataset>
Dimensions:  (lat: 24, lon: 72, mode: 100, month: 12, year: 100)
Coordinates:
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * mode     (mode) int64 10000 10000 10000 10000 ... 10000 10000 10000 10000
    time     (year, month) int32 1 2 3 4 5 6 7 ... 1195 1196 1197 1198 1199 1200
  * year     (year) int64 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    sst      (lat, lon, year, month) float32 1.8618509 1.2667823 ... -0.01160882
    t300     (lat, lon, year, month) float32 0.3742796 0.32198605 ... 0.02676518
    ua       (lat, lon, year, month) float64 0.4293 1.043 ... -2.384 1.878
    va       (lat, lon, year, month) float64 0.08719 0.1337 ... -1.478 -2.785
    nino     (year, month) float64 -0.4072 -0.2024 -0.1039 ... -1.177 -1.365

In [392]:
xr.combine_by_coords(SODA)

AttributeError: 'str' object has no attribute 'dims'

In [391]:
xr.concat([CMIP,SODA],'mode')

ValueError: cannot reindex or align along dimension 'year' because the index has duplicate values